In [56]:
import requests
from bs4 import BeautifulSoup as bs
import re

In [57]:
# Get_Beautifulsoup
PMID = "17942706 "
URL = f"https://pubmed.ncbi.nlm.nih.gov/{PMID}/"

r = requests.get(URL)
print(f"Status code: {r.status_code}")
html_pubmed = bs(r.text, 'html.parser')


Status code: 200


In [58]:
# Get_year_of_pub
year = int(html_pubmed.find('div', {'class': "article-source"}).find('span', {'class': "cit"}).text.split(' ', 1)[0])
year_dict = {'year_of_publication': year}
print(year_dict)

{'year_of_publication': 2007}


In [59]:
# Get_PMCID
try:
    PMCID = html_pubmed.find('span', {'class': "identifier pmc"}).a.text.strip()
except AttributeError:
    print("No PMCID")
else:
    PMCID_dict = {'PMCID': PMCID}
print(PMCID_dict)

{'PMCID': 'PMC2045134'}


In [60]:
# Get_DOI
DOI = html_pubmed.find('span', {'class': "identifier doi"}).a.text.strip()
DOI_dict ={'DOI': DOI}
print(DOI_dict)

{'DOI': '10.1101/gad.1604007'}


In [61]:
# Get_PMID
PMID = html_pubmed.find('strong', {'class': "current-id"}).text.strip()
PMID_dict ={'PMID': PMID}
print(PMID_dict)

{'PMID': '17942706'}


In [62]:
# Get_title
title = html_pubmed.h1.text.strip()
title_dict = {'title': title}
print(title_dict)


{'title': 'Phosphorylation of histone H3 at Ser10 facilitates RNA polymerase II release from promoter-proximal pausing in Drosophila'}


In [63]:
# Get_authors_and_affiliations
auteurs = []
affiliations = []
authors = html_paper.find_all('span', {'class': "authors-list-item"})
for author in authors:
    try:
        aut = author.find('a', {'class': "full-name"})['data-ga-label']
        auteurs.append(aut)
        affil = author.find('a', {'class': "affiliation-link"})['title']
        affiliations.append(affil)
    except TypeError:
        continue

authors_affiliation_dict = dict(zip(auteurs, affiliations))
print(authors_affiliation_dict)



{'Bernd Stadelmayer': 'INSERM U1052, CNRS UMR-5286, Cancer Research Center of Lyon (CRCL), Lyon, 69008, France; University of Lyon, Université Claude-Bernard (UCBL), 69008 Lyon, France. Electronic address: bernd.stadelmayer@inserm.fr.', 'Audrey Diederichs': 'INSERM U1052, CNRS UMR-5286, Cancer Research Center of Lyon (CRCL), Lyon, 69008, France; University of Lyon, Université Claude-Bernard (UCBL), 69008 Lyon, France.', 'Fleur Chapus': 'INSERM U1052, CNRS UMR-5286, Cancer Research Center of Lyon (CRCL), Lyon, 69008, France; University of Lyon, Université Claude-Bernard (UCBL), 69008 Lyon, France.', 'Michel Rivoire': 'INSERM U1032, Centre Léon Bérard (CLB), 69008 Lyon, France.', 'Gregory Neveu': "Evotec, 1541 Avenue Marcel Mérieux, 69280 Marcy l'Etoile, France.", 'Antoine Alam': "Evotec, 1541 Avenue Marcel Mérieux, 69280 Marcy l'Etoile, France.", 'Laurent Fraisse': "Evotec, 1541 Avenue Marcel Mérieux, 69280 Marcy l'Etoile, France.", 'Kara Carter': "Evotec, 1541 Avenue Marcel Mérieux, 69

In [64]:
# Get_Abstract
abstract = html_pubmed.find('div', {'class': "abstract-content selected"}).text.strip().replace('\n', '')
abstract_dict = {'abstract': abstract}
print(abstract_dict)


{'abstract': 'The Drosophila JIL-1 kinase is known to phosphorylate histone H3 at Ser10 (H3S10) during interphase. This modification is associated with transcriptional activation, but its function is not well understood. Here we present evidence suggesting that JIl-1-mediated H3S10 phosphorylation is dependent on chromatin remodeling by the brahma complex and is required during early transcription elongation to release RNA polymerase II (Pol II) from promoter-proximal pausing. JIL-1 localizes to transcriptionally active regions and is required for activation of the E75A ecdysone-responsive and hsp70 heat-shock genes. The heat-shock transcription factor, the promoter-paused form of Pol II (Pol IIo(ser5)), and the pausing factor DSIF (DRB sensitivity-inducing factor) are still present at the hsp70 loci in JIL-1-null mutants, whereas levels of the elongating form of Pol II (Pol IIo(ser2)) and the P-TEFb kinase are dramatically reduced. These observations suggest that phosphorylation of H3

In [65]:
# GetHTML_Pubmed_Citedby

URL = "https://pubmed.ncbi.nlm.nih.gov/?linkname=pubmed_pubmed_citedin&from_uid=" + f"{PMID}"

r = requests.get(URL)
print(f"Status code: {r.status_code}")
citedby = bs(r.text, 'html.parser')

Status code: 200


In [66]:
# GetNumberPages
ref_citing =[]
try:
    number_total_pages = int(citedby.find('label', {'class': "of-total-pages"}).text.split(' ')[1])
except AttributeError:
    print("Not cited")
else:
    # Get RefCytedBy
    
    ref_page_1 = citedby.find_all('span', {'class': "docsum-journal-citation full-journal-citation"})
    
    for doi in ref_page_1:
        ref_doi = doi.text.split('doi:')[1].lstrip(' ')
        if len(ref_doi.split(' ')) > 1:
            ref_citing.append(ref_doi.split(' ')[0].rstrip('.'))
        else:
            ref_citing.append(ref_doi.rstrip('.'))


if number_total_pages > 1:
    for i in range(2, (number_total_pages +1)):
        URL_suite = f"{URL}&page={i}"
        
        r = requests.get(URL_suite)
        print(f"Status code: {r.status_code}")
        citedby_suite = bs(r.text, 'html.parser')
            
        ref_pages_suite = citedby_suite.find_all('span', {'class': "docsum-journal-citation full-journal-citation"})
        for doi in ref_pages_suite:
            ref_doi = doi.text.split('doi:')[1].lstrip(' ')
            if len(ref_doi.split(' ')) > 1:
                ref_citing.append(ref_doi.split(' ')[0].rstrip('.'))
            else:
                ref_citing.append(ref_doi.rstrip('.'))
del ref_citing[0]

Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200
Status code: 200


In [67]:
print(ref_citing)
print(len(ref_citing))

['10.3390/genes12101596', '10.3390/ijms22073458', '10.1038/s43018-020-00171-8', '10.1038/s41467-020-20089-0', '10.18632/aging.102684', '10.1007/s00412-019-00728-2', '10.1016/j.celrep.2019.07.096', '10.1186/s12864-017-4245-x', '10.1242/dev.151134', '10.1105/tpc.17.00266', '10.3390/genes8050142', '10.1007/s00412-016-0617-2', '10.1016/j.tig.2016.10.002', '10.2174/1570162x14666160324124736', '10.1186/s13578-016-0081-y', '10.1007/s13365-015-0398-z', '10.1139/bcb-2015-0045', '10.1038/nrg3961', '10.1016/j.virol.2015.03.036', '10.1101/cshperspect.a019398', '10.1002/path.4489', '10.5483/bmbrep.2014.47.4.060', '10.1093/nar/gku173', '10.1242/dev.088492', '10.1016/j.molcel.2014.01.005', '10.1093/nar/gkt722', '10.1016/B978-0-12-407708-9.00002-9', '10.4161/cc.24992', '10.1371/journal.pone.0062484', '10.1186/1471-213X-13-7', '10.1128/MCB.00323-12', '10.1371/journal.pone.0053091', '10.1371/journal.pgen.1003153', '10.1074/jbc.M112.427336', '10.3390/ijms131012461', '10.1101/gr.136788.111', '10.1371/jour